In [12]:
import numpy as np
import pandas as pd
import math

In [13]:
similarity_matrix = ['similarity_smallcate.csv','similarity_bigcate.csv','similarity_time_space.csv','similarity_place.csv']
dis_prefer = pd.read_csv('distance_preference.csv')
checkin_times = pd.read_csv('CheckInTimes.csv')

In [16]:
def simi_user(user, n, matrix_no):
    # find top n similar user
    # return user id

    # read similarity_matrix
    similarity = pd.read_csv(similarity_matrix[matrix_no])
    similarity.index = range(1, len(similarity) + 1)
    # find similar users
    user_row = pd.DataFrame(similarity.loc[user])
    sort = pd.DataFrame.sort_values(user_row, by=user, ascending=False)
    return [int(i) for i in sort[:n].index]


def already_been(user):
    return checkin_times.loc[checkin_times.userId == user]


def candidate_place(user, n, matrix_no):
    s_users = simi_user(user, n, matrix_no)
    candidates = pd.DataFrame(columns=('venueId', 'venueCategory', 'latitude', 'longitude', 'CheckInCount'))
    for i in s_users:
        candidates = candidates.append(checkin_times.loc[checkin_times.userId == i][
                                           ['venueId', 'venueCategory', 'latitude', 'longitude', 'CheckInCount']])
    new = pd.DataFrame(columns=('venueId', 'venueCategory', 'latitude', 'longitude', 'CheckInCount'))
    for i in candidates.venueId.unique():
        temp = candidates[candidates.venueId == i]
        new = new.append(
            {'venueId': i, 'venueCategory': list(temp.venueCategory)[0], 'latitude': list(temp.latitude)[0],
             'longitude': list(temp.longitude)[0], 'CheckInCount': np.sum(temp.CheckInCount)}, ignore_index=True)
    # delete places already been
    already = already_been(user)
    feasible_place = new.loc[~new.venueId.isin(already['venueId'])]
    feasible_place = pd.DataFrame.sort_values(feasible_place, ['CheckInCount'], ascending=False)
    return feasible_place


def distance(lat0, lon0, lat1, lon1):
    return np.sqrt((lat1 - lat0) ** 2 * (111 * np.cos(lat1)) ** 2 + (lon1 - lon0) ** 2 * 111 ** 2)


def recommend(user, topN, neighbours, matrix_no):
    final_choice = pd.DataFrame(columns=('venueId', 'venueCategory', 'latitude', 'longitude', 'distance'))
    dist = dis_prefer.loc[user]
    lat0, lon0, aver = dist.latitude, dist.longitude, dist.avgDistanceTravel
    places = candidate_place(user, neighbours, matrix_no)
    for i in list(places.index):
        if len(final_choice) == topN:
            final_choice.drop(columns=['CheckInCount'], inplace=True)
            final_choice.index = range(1, len(final_choice) + 1)
            return final_choice
        p = places.loc[i]
        lat1 = p.latitude
        lon1 = p.longitude
        km = distance(lat0, lon0, lat1, lon1)
        if km < aver:
            final_choice = final_choice.append(p)
            final_choice.loc[p.name, 'distance'] = km
    if len(final_choice) == 0:
        return 'Can not find any new place to go.'
    else:
        final_choice.drop(columns=['CheckInCount'], inplace=True)
        final_choice.index = range(1, len(final_choice) + 1)
        return final_choice

In [17]:
recommend(3,5,10,0)

,venueId,venueCategory,latitude,longitude,distance
1,4c787720dbaa76b0144f154b,Train Station,35.666963,139.758378,0.953398
2,4efd56c06c25c5ce7ee7f759,Hotel,35.698355,139.749599,0.919288
3,4b582e2bf964a5201a4d28e3,Park,35.683832,139.744592,0.822094
4,4b7bee49f964a5205c742fe3,Historic Site,35.683750,139.745604,0.713395
5,4c505692d49d76b0d4b2187b,Road,35.689666,139.751442,0.496750


In [18]:
recommend(3,5,10,1)

,venueId,venueCategory,latitude,longitude,distance
1,4b5e638ef964a520a58c29e3,Park,35.666383,139.752533,0.637988
2,4b600990f964a520e3d329e3,Train Station,35.666692,139.758133,0.941349
3,4c787720dbaa76b0144f154b,Train Station,35.666963,139.758378,0.953398
4,4b6a5612f964a52088d22be3,Fast Food Restaurant,35.667543,139.757322,0.845500
5,4de0b8a552b1c1b3ce39b9dc,Japanese Restaurant,35.664970,139.755471,0.817484


In [19]:
recommend(3,5,10,2)

,venueId,venueCategory,latitude,longitude,distance
1,4ba9be42f964a5207e363ae3,Office,35.700362,139.750428,0.987617
2,4b600990f964a520e3d329e3,Train Station,35.666692,139.758133,0.941349
3,4b835fdff964a5204e0431e3,Convenience Store,35.700734,139.750776,0.998990
4,4b46d0ebf964a520982826e3,Subway,35.695504,139.751705,0.761419
5,4e0333e4e4cdbd9a517691ce,Community College,35.676925,139.761125,1.049582


In [20]:
recommend(3,5,10,3)

,venueId,venueCategory,latitude,longitude,distance
1,4b600990f964a520e3d329e3,Train Station,35.666692,139.758133,0.941349
2,4b5ab0ddf964a520f5d028e3,Stadium,35.693204,139.749827,0.690288
3,4b443147f964a5208bf225e3,Train Station,35.695966,139.758067,1.052735
4,4d5216eec5ff6ea834feaa07,Ramen / Noodle House,35.701185,139.750235,1.026571
5,4b05879cf964a520549c22e3,Hotel,35.666867,139.743948,1.054748
